In [0]:
import pyspark.sql.functions as sf

In [0]:
%sql
create schema if not exists nyctaxi_copy
LOCATION 'dbfs:/user/hive/warehouse';

use schema nyctaxi_copy;

create or replace table trips as (
  select * from samples.nyctaxi.trips
);

num_affected_rows,num_inserted_rows


In [0]:
trips_df = spark.read.format("delta").table("nyctaxi_copy.trips")

pickup_times_df = (
    trips_df.select(sf.col("tpep_pickup_datetime"))
    .dropDuplicates()
)

while pickup_times_df.isEmpty() == False:
    min_pu_time_df = (
        pickup_times_df
        .select(
            sf.min(sf.col("tpep_pickup_datetime")).alias("min_pu_datetime")
        )
    )

    min_pu_time = [
        i[0] for i in 
        min_pu_time_df.select("min_pu_datetime").collect()
    ][0]

    trips_df_subset = (
        trips_df
        .where(sf.col("tpep_pickup_datetime") == min_pu_time)
        .withColumn("streaming_timestamp", sf.current_timestamp())
    )

    pickup_times_df = (
        pickup_times_df
        .where(sf.col("tpep_pickup_datetime") != min_pu_time)
    )

    trips_df_subset.write.mode("append").saveAsTable("nyctaxi_copy.trips_src")